In [1]:
# Packges
import requests
from bs4 import BeautifulSoup

In [2]:
# Requesting a html response from the Home page of Installation guide
url = 'https://packaging.python.org/en/latest/guides/section-install/'
response = requests.get(url)

# response check
response.status_code

200

In [3]:
# Extracting links from Homepage and storing them as a list
soup = BeautifulSoup(response.content, 'lxml' )
ins = soup.find(id='installation')
ul = ins.find('ul')
a_tags = ul.find_all('a')
pages=[]
for a_tag in a_tags:
    page = a_tag.get('href')
    pages.append(page)
    
links = [link[link.find('installing'):] for link in pages]
links

['installing-using-pip-and-virtual-environments/',
 'installing-using-virtualenv/',
 'installing-stand-alone-command-line-tools/',
 'installing-using-linux-tools/',
 'installing-scientific-packages/']

In [10]:
# Looping through each page
i=0
for link in links:
    i += 1
    steps=[]
    url = f"https://packaging.python.org/en/latest/guides/{link}" 
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    # Grabbing main html containing all the necessary tags
    main_sec = soup.select_one('section')
    sections = main_sec.select('section')
    
    
    # Function to store data in a list
    def store(s):
        steps.append(s + '\n')

        
    # Function to check whether the neccessary code block exists
    def code_parse(codes):
        if(codes!=[]):
            for code in codes:
                c = f"Code: {code.text}"
                store(c)
        else:
            pass
        
        
    # Function traversing the html tree for Headings and Sub_headings
    '''
    Logic Steps:-
    1) fn checks for Heading
    2) fn checks for further child sections if not found checks for code block
    3) fn checks for Sub-Heading
    4) fn checks for code again
    '''
    def h_tags(h):
        h2 = h.select_one('h2')
        if(h2):
            s2 = f"\nHeading: {h2.text.rstrip('#')}."
            store(s2)            
            sections = h.select('section')
            if(sections!=[]):
                for section in sections:
                    h3 = section.select_one('h3')
                    if(h3):
                        s3=f"Sub-Heading: {h3.text.rstrip('#')}."
                        store(s3)
                        c = section.find_all('div', class_='highlight-bash notranslate')
                        code_parse(c)
            else:
                c = h.find_all('div', class_='highlight-bash notranslate')
                code_parse(c) 

        
            


    # Looping through primary sections in the tree if it consists more than one 
    if(len(sections)>1):
        for section in sections:
            h_tags(section)
    else:
        h1 = main_sec.select_one('h1')
        head = f"Heading: {h1.text.rstrip('#')}"
        store(head)
        h1_code = main_sec.find_all('div', class_='highlight-bash notranslate')
        code_parse(h1_code)


        
    print(steps)
    filename = f'Step{i}.csv'

# Writing each step of installation guide into separate files according to their respective position at home page
    with open(filename, 'w') as f:
        for step in steps:
            f.write(step + '\n')

['\nHeading: Create and Use Virtual Environments.\n', 'Sub-Heading: Create a new virtual environment.\n', 'Code: python3 -m venv .venv\n\n\n', 'Sub-Heading: Activate a virtual environment.\n', 'Code: source .venv/bin/activate\n\n\n', 'Code: which python\n\n\n', 'Code: .venv/bin/python\n\n\n', 'Sub-Heading: Deactivate a virtual environment.\n', 'Code: deactivate\n\n\n', 'Sub-Heading: Reactivate a virtual environment.\n', '\nHeading: Prepare pip.\n', 'Code: python3 -m pip install --upgrade pip\npython3 -m pip --version\n\n\n', '\nHeading: Install packages using pip.\n', 'Sub-Heading: Install a package.\n', 'Code: python3 -m pip install requests\n\n\n', 'Sub-Heading: Install a specific package version.\n', "Code: python3 -m pip install 'requests==2.18.4'\n\n\n", "Code: python3 -m pip install 'requests>=2.0.0,<3.0.0'\n\n\n", 'Code: python3 -m pip install --pre requests\n\n\n', 'Sub-Heading: Install extras.\n', "Code: python3 -m pip install 'requests[security]'\n\n\n", 'Sub-Heading: Install